# Model Demo

In [6]:
import numpy as np
import torch
from torch.backends import cudnn
import cv2
import torchvision
from PIL import Image
import matplotlib.pyplot as plt

In [10]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

GeForce GTX 1650


### Utility Functions
Functions to transform, get mask predictions and apply masks on the input images.<br />
<hr>

In [7]:
def infer_op(op):
'''convert output to cpu tensor then to numpy array and scale up range from [0.0 to 1.0] to [0 to 255]'''
    op = op.cpu().numpy()
    op = op * 255
    op.astype(np.uint8)

    return op

def apply_mask(img, mask):
'''Apply generated mask on image'''
    assert np.shape(mask) == (224, 224)

    img = cv2.resize(img, (224, 224))
    portrait = cv2.bitwise_and(img, img, mask = mask)

    return portrait

In [9]:
def get_mask(img, model):
'''Transform input image to make compatible with model and return mask image'''
    #CUDA for PyTorch
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    cudnn.benchmark = True

    model.to(device)
    model.eval()

    tf = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=True)
    ])
    img = tf(img).to(device)

    mask = model(img)

    return infer_op(mask)

<hr>

## Applying Model
Apply model on custom image.<br /> `img_path` refers to location of image.

In [ ]:
img_path = ''

Load pretrained model

In [ ]:
model = torch.load('../models/deeplab.pt')

### Input Image

In [ ]:
img = Image.open(img_path)
plt.imshow(img)

Resizing image to 224x224 to make it compatible with model input.

In [ ]:
inp_size = (224, 224)
img = img.resize(inp_size)

### Generated Mask

In [ ]:
mask = get_mask(img, model)
plt.imshow(mask, 'gray')

### Portrait
Mask applied on input image.

In [ ]:
portrait = apply_mask(img, mask)
plt.imshow(portrait)